In [1]:
# Mount Google Drive to access your dataset
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted successfully!")
print("You can now access files in: /content/drive/MyDrive/")



Mounted at /content/drive
✅ Google Drive mounted successfully!
You can now access files in: /content/drive/MyDrive/


In [2]:
# Install all necessary packages for FinBERT fine-tuning
!pip install transformers torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install datasets accelerate evaluate scikit-learn pandas matplotlib seaborn

print("\n✅ All packages installed successfully!")


Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00

✅ All packages installed successfully!


In [3]:
# Import all necessary libraries
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Transformers and Hugging Face
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import evaluate

print("✅ All libraries imported successfully!")




✅ All libraries imported successfully!


In [4]:
# Define paths for your dataset and model
# Update these paths according to where your files are stored


DATASET_PATH = "/content/drive/MyDrive/merged_financial_sentiment.json"  # Update this path



# Model name (FinBERT)
MODEL_NAME = "ProsusAI/finbert"  # Pre-trained FinBERT model

# Output directory for saving fine-tuned model
OUTPUT_DIR = "/content/drive/MyDrive/finetuned_finbert"  # Update this if needed

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"✅ Paths configured!")
print(f"Dataset path: {DATASET_PATH}")
print(f"Model: {MODEL_NAME}")
print(f"Output directory: {OUTPUT_DIR}")

✅ Paths configured!
Dataset path: /content/drive/MyDrive/merged_financial_sentiment.json
Model: ProsusAI/finbert
Output directory: /content/drive/MyDrive/finetuned_finbert


In [5]:
# Check if dataset file exists
if os.path.exists(DATASET_PATH):
    print(f"✅ Dataset found at: {DATASET_PATH}")

    # Check file size
    file_size = os.path.getsize(DATASET_PATH) / (1024 * 1024)  # Size in MB
    print(f"File size: {file_size:.2f} MB")
else:
    print(f"❌ Dataset not found at: {DATASET_PATH}")
    print("Please update the DATASET_PATH in the previous cell.")




✅ Dataset found at: /content/drive/MyDrive/merged_financial_sentiment.json
File size: 3.28 MB


In [6]:
# Load JSON dataset
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

def load_json_dataset(json_path):
    """Load JSON dataset (supports both JSON and JSONL formats)."""
    print(f"Loading dataset from: {json_path}")

    # Try to detect JSONL format (JSON Lines - one JSON object per line)
    data = []
    with open(json_path, 'r', encoding='utf-8') as f:
        first_line = f.readline().strip()
        f.seek(0)  # Reset to beginning

        # Check if it's JSONL format
        if first_line.startswith('{') and not first_line.startswith('[{'):
            # JSONL format - read line by line
            print("Detected JSONL format (JSON Lines)")
            for line in f:
                line = line.strip()
                if line:  # Skip empty lines
                    try:
                        data.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(f"Warning: Skipping invalid JSON line: {e}")
        else:
            # Standard JSON format
            data = json.load(f)

    # Convert to DataFrame
    if isinstance(data, list):
        df = pd.DataFrame(data)
    elif isinstance(data, dict):
        # If it's a dict, try to find the key that contains the list
        for key in ['data', 'records', 'items', 'samples']:
            if key in data and isinstance(data[key], list):
                df = pd.DataFrame(data[key])
                break
        else:
            df = pd.DataFrame([data])
    else:
        raise ValueError("JSON format not supported.")

    print(f"✅ Dataset loaded successfully!")
    print(f"Dataset shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"\nFirst few rows:")
    print(df.head())

    return df

# Load your dataset
# Update these paths and column names according to your dataset
DATASET_PATH = "/content/drive/MyDrive/merged_financial_sentiment.json"  # Update this
TEXT_COLUMN = "text"  # Update this if your text column has a different name
LABEL_COLUMN = "label"  # Update this if your label column has a different name

df = load_json_dataset(DATASET_PATH)

# Display label distribution
print("\n" + "=" * 60)
print("LABEL DISTRIBUTION")
print("=" * 60)
print(df[LABEL_COLUMN].value_counts().sort_index())

Loading dataset from: /content/drive/MyDrive/merged_financial_sentiment.json
Detected JSONL format (JSON Lines)
✅ Dataset loaded successfully!
Dataset shape: (22989, 2)
Columns: ['text', 'label']

First few rows:
                                                text  label
0  According to Gran , the company has no plans t...      1
1  For the last quarter of 2010 , Componenta 's n...      2
2  In the third quarter of 2010 , net sales incre...      2
3  Operating profit rose to EUR 13.1 mn from EUR ...      2
4  Operating profit totalled EUR 21.1 mn , up fro...      2

LABEL DISTRIBUTION
label
0     3412
1     7989
2    11588
Name: count, dtype: int64


In [7]:
# Split dataset into train, validation, and test sets (80:10:10)
from sklearn.model_selection import train_test_split

def split_dataset(df, text_column, label_column, test_size=0.1, val_size=0.1, random_state=42):
    """
    Split dataset into train, validation, and test sets.

    Parameters:
    - df: DataFrame
    - text_column: name of text column
    - label_column: name of label column
    - test_size: proportion of test set (default 0.1 = 10%)
    - val_size: proportion of validation set from remaining data (default 0.1 = 10% of total)
    - random_state: random seed for reproducibility

    Returns:
    - train_df, val_df, test_df
    """

    # First split: separate test set (10%)
    # This leaves 90% for train+val
    df_temp, test_df = train_test_split(
        df,
        test_size=test_size,
        random_state=random_state,
        stratify=df[label_column]  # Maintain class distribution
    )

    # Second split: separate validation set from remaining data
    # We want val_size of TOTAL, so from the 90% remaining:
    # val_size / (1 - test_size) = 0.1 / 0.9 ≈ 0.111
    val_proportion = val_size / (1 - test_size)

    train_df, val_df = train_test_split(
        df_temp,
        test_size=val_proportion,
        random_state=random_state,
        stratify=df_temp[label_column]  # Maintain class distribution
    )

    # Print split information
    print("=" * 60)
    print("DATASET SPLIT (80:10:10)")
    print("=" * 60)
    print(f"Total samples: {len(df)}")
    print(f"\nTrain set:   {len(train_df):6d} samples ({len(train_df)/len(df)*100:.1f}%)")
    print(f"Val set:     {len(val_df):6d} samples ({len(val_df)/len(df)*100:.1f}%)")
    print(f"Test set:    {len(test_df):6d} samples ({len(test_df)/len(df)*100:.1f}%)")

    print("\n" + "-" * 60)
    print("CLASS DISTRIBUTION IN EACH SPLIT:")
    print("-" * 60)

    print("\nTrain set:")
    print(train_df[label_column].value_counts().sort_index())

    print("\nValidation set:")
    print(val_df[label_column].value_counts().sort_index())

    print("\nTest set:")
    print(test_df[label_column].value_counts().sort_index())

    return train_df, val_df, test_df

# Split the dataset
train_df, val_df, test_df = split_dataset(
    df,
    text_column=TEXT_COLUMN,
    label_column=LABEL_COLUMN,
    test_size=0.1,
    val_size=0.1,
    random_state=42
)

DATASET SPLIT (80:10:10)
Total samples: 22989

Train set:    18391 samples (80.0%)
Val set:       2299 samples (10.0%)
Test set:      2299 samples (10.0%)

------------------------------------------------------------
CLASS DISTRIBUTION IN EACH SPLIT:
------------------------------------------------------------

Train set:
label
0    2730
1    6391
2    9270
Name: count, dtype: int64

Validation set:
label
0     341
1     799
2    1159
Name: count, dtype: int64

Test set:
label
0     341
1     799
2    1159
Name: count, dtype: int64


In [8]:
# Load FinBERT tokenizer
MODEL_NAME = "ProsusAI/finbert"

print(f"Loading tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print("✅ Tokenizer loaded successfully!")
print(f"Tokenizer vocab size: {tokenizer.vocab_size}")
print(f"Max model input length: {tokenizer.model_max_length}")

# Check tokenizer special tokens
print(f"\nSpecial tokens:")
print(f"  Padding token: {tokenizer.pad_token}")
print(f"  CLS token: {tokenizer.cls_token}")
print(f"  SEP token: {tokenizer.sep_token}")
print(f"  UNK token: {tokenizer.unk_token}")

Loading tokenizer: ProsusAI/finbert


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

✅ Tokenizer loaded successfully!
Tokenizer vocab size: 30522
Max model input length: 512

Special tokens:
  Padding token: [PAD]
  CLS token: [CLS]
  SEP token: [SEP]
  UNK token: [UNK]


In [9]:
# Tokenize the datasets
def tokenize_function(examples, text_column, tokenizer, max_length=512):
    """
    Tokenize the text data.

    Parameters:
    - examples: dictionary containing text data
    - text_column: name of the text column
    - tokenizer: tokenizer object
    - max_length: maximum sequence length

    Returns:
    - Tokenized examples
    """
    return tokenizer(
        examples[text_column],
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors=None  # Return lists, not tensors
    )

# Convert DataFrames to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

print("✅ Datasets converted to Hugging Face format")
print(f"Train dataset: {len(train_dataset)} samples")
print(f"Val dataset: {len(val_dataset)} samples")
print(f"Test dataset: {len(test_dataset)} samples")

# Tokenize all datasets
MAX_LENGTH = 512  # Adjust based on your data and model requirements

print(f"\nTokenizing datasets with max_length={MAX_LENGTH}...")
train_tokenized = train_dataset.map(
    lambda x: tokenize_function(x, TEXT_COLUMN, tokenizer, MAX_LENGTH),
    batched=True,
    remove_columns=train_dataset.column_names  # Remove original columns
)

val_tokenized = val_dataset.map(
    lambda x: tokenize_function(x, TEXT_COLUMN, tokenizer, MAX_LENGTH),
    batched=True,
    remove_columns=val_dataset.column_names
)

test_tokenized = test_dataset.map(
    lambda x: tokenize_function(x, TEXT_COLUMN, tokenizer, MAX_LENGTH),
    batched=True,
    remove_columns=test_dataset.column_names
)

print("✅ Tokenization completed!")

✅ Datasets converted to Hugging Face format
Train dataset: 18391 samples
Val dataset: 2299 samples
Test dataset: 2299 samples

Tokenizing datasets with max_length=512...


Map:   0%|          | 0/18391 [00:00<?, ? examples/s]

Map:   0%|          | 0/2299 [00:00<?, ? examples/s]

Map:   0%|          | 0/2299 [00:00<?, ? examples/s]

✅ Tokenization completed!


In [13]:
#This is the one I am using
# Tokenize datasets but keep the label column
train_tokenized = train_dataset.map(
    lambda x: tokenize_function(x, TEXT_COLUMN, tokenizer, MAX_LENGTH),
    batched=True,
    batch_size=1000,
    remove_columns=[col for col in train_dataset.column_names if col != LABEL_COLUMN]  # Keep label column
)

val_tokenized = val_dataset.map(
    lambda x: tokenize_function(x, TEXT_COLUMN, tokenizer, MAX_LENGTH),
    batched=True,
    batch_size=1000,
    remove_columns=[col for col in val_dataset.column_names if col != LABEL_COLUMN]
)

test_tokenized = test_dataset.map(
    lambda x: tokenize_function(x, TEXT_COLUMN, tokenizer, MAX_LENGTH),
    batched=True,
    batch_size=1000,
    remove_columns=[col for col in test_dataset.column_names if col != LABEL_COLUMN]
)

# Prepare labels and convert to IDs
train_labels, label_to_id = prepare_labels(train_df, LABEL_COLUMN)
val_labels, _ = prepare_labels(val_df, LABEL_COLUMN, label_to_id)
test_labels, _ = prepare_labels(test_df, LABEL_COLUMN, label_to_id)

# Replace the label column with numeric IDs
train_tokenized = train_tokenized.remove_columns([LABEL_COLUMN]).add_column("labels", train_labels)
val_tokenized = val_tokenized.remove_columns([LABEL_COLUMN]).add_column("labels", val_labels)
test_tokenized = test_tokenized.remove_columns([LABEL_COLUMN]).add_column("labels", test_labels)

Map:   0%|          | 0/18391 [00:00<?, ? examples/s]

Map:   0%|          | 0/2299 [00:00<?, ? examples/s]

Map:   0%|          | 0/2299 [00:00<?, ? examples/s]

Label mapping: {np.int64(0): 0, np.int64(1): 1, np.int64(2): 2}
Number of classes: 3


In [15]:
# Define path for saving tokenized datasets
TOKENIZED_DATA_DIR = "/content/drive/MyDrive/finetuned_finbert/tokenized_datasets"
os.makedirs(TOKENIZED_DATA_DIR, exist_ok=True)

# Create DatasetDict and save to Google Drive
dataset_dict = DatasetDict({
    "train": train_tokenized,
    "validation": val_tokenized,
    "test": test_tokenized
})

dataset_dict.save_to_disk(TOKENIZED_DATA_DIR)
print(f"✅ Tokenized datasets saved to: {TOKENIZED_DATA_DIR}")

# Save label mapping
import pickle
with open(f"{TOKENIZED_DATA_DIR}/label_to_id.pkl", 'wb') as f:
    pickle.dump(label_to_id, f)
print(f"✅ Label mapping saved")

Saving the dataset (0/1 shards):   0%|          | 0/18391 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2299 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2299 [00:00<?, ? examples/s]

✅ Tokenized datasets saved to: /content/drive/MyDrive/finetuned_finbert/tokenized_datasets
✅ Label mapping saved


In [16]:
# Load FinBERT model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

MODEL_NAME = "ProsusAI/finbert"

print(f"Loading model: {MODEL_NAME}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load label mapping to get number of classes
import pickle
TOKENIZED_DATA_DIR = "/content/drive/MyDrive/finetuned_finbert/tokenized_datasets"
with open(f"{TOKENIZED_DATA_DIR}/label_to_id.pkl", 'rb') as f:
    label_to_id = pickle.load(f)

NUM_LABELS = len(label_to_id)
print(f"Number of classes: {NUM_LABELS}")
print(f"Label mapping: {label_to_id}")

# Load model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    problem_type="single_label_classification"  # For multi-class classification
)

print("✅ Model loaded successfully!")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

Loading model: ProsusAI/finbert
Number of classes: 3
Label mapping: {np.int64(0): 0, np.int64(1): 1, np.int64(2): 2}


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

✅ Model loaded successfully!
Model parameters: 109,484,547
Trainable parameters: 109,484,547


In [30]:
# Training Configuration and Hyperparameters
TRAINING_CONFIG = {
    # Model paths
    "output_dir": "/content/drive/MyDrive/finetuned_finbert/model_output",
    "logging_dir": "/content/drive/MyDrive/finetuned_finbert/logs",

    # Hyperparameters
    "learning_rate": 2e-5,  # Typical range: 2e-5 to 5e-5 for transformers
    "num_train_epochs": 5,  # Adjust based on your dataset size and needs
    "per_device_train_batch_size": 16,  # Adjust based on GPU memory (8, 16, 32)
    "per_device_eval_batch_size": 16,
    "weight_decay": 0.01,  # L2 regularization
    "warmup_steps": 500,  # Number of warmup steps for learning rate scheduler
    "warmup_ratio": 0.1,  # Alternative to warmup_steps (ratio of total training steps)

    # Learning rate schedule
    "lr_scheduler_type": "linear",  # Options: linear, cosine, cosine_with_restarts, polynomial, constant, constant_with_warmup

    # Optimizer
    "optim": "adamw_torch",  # AdamW optimizer

    # Training settings
    "logging_steps": 100,  # Log every N steps
    "eval_strategy": "epoch",  # Evaluate after each epoch (options: "steps", "epoch", "no")
    "save_strategy": "epoch",  # Save checkpoint after each epoch
    "save_total_limit": 3,  # Keep only the last 3 checkpoints
    "load_best_model_at_end": True,  # Load best model at the end
    "metric_for_best_model": "f1",  # Metric to use for best model selection
    "greater_is_better": True,  # Whether higher metric is better

    # Other settings
    "seed": 42,  # For reproducibility
    "fp16": True,  # Use mixed precision training (faster, less memory)
    "dataloader_num_workers": 2,  # Number of workers for data loading
    "report_to": "tensorboard",  # Logging: tensorboard, wandb, etc.
    "push_to_hub": False,  # Set to True if you want to push to Hugging Face Hub
}

print("=" * 60)
print("TRAINING CONFIGURATION")
print("=" * 60)
for key, value in TRAINING_CONFIG.items():
    print(f"{key}: {value}")
print("=" * 60)

TRAINING CONFIGURATION
output_dir: /content/drive/MyDrive/finetuned_finbert/model_output
logging_dir: /content/drive/MyDrive/finetuned_finbert/logs
learning_rate: 2e-05
num_train_epochs: 5
per_device_train_batch_size: 16
per_device_eval_batch_size: 16
weight_decay: 0.01
warmup_steps: 500
warmup_ratio: 0.1
lr_scheduler_type: linear
optim: adamw_torch
logging_steps: 100
eval_strategy: epoch
save_strategy: epoch
save_total_limit: 3
load_best_model_at_end: True
metric_for_best_model: f1
greater_is_better: True
seed: 42
fp16: True
dataloader_num_workers: 2
report_to: tensorboard
push_to_hub: False


In [31]:
# Create TrainingArguments object
training_args = TrainingArguments(
    output_dir=TRAINING_CONFIG["output_dir"],
    logging_dir=TRAINING_CONFIG["logging_dir"],

    # Hyperparameters
    learning_rate=TRAINING_CONFIG["learning_rate"],
    num_train_epochs=TRAINING_CONFIG["num_train_epochs"],
    per_device_train_batch_size=TRAINING_CONFIG["per_device_train_batch_size"],
    per_device_eval_batch_size=TRAINING_CONFIG["per_device_eval_batch_size"],
    weight_decay=TRAINING_CONFIG["weight_decay"],
    warmup_steps=TRAINING_CONFIG["warmup_steps"],

    # Learning rate scheduler
    lr_scheduler_type=TRAINING_CONFIG["lr_scheduler_type"],

    # Optimizer
    optim=TRAINING_CONFIG["optim"],

    # Training settings
    logging_steps=TRAINING_CONFIG["logging_steps"],
    eval_strategy=TRAINING_CONFIG["eval_strategy"],
    save_strategy=TRAINING_CONFIG["save_strategy"],
    save_total_limit=TRAINING_CONFIG["save_total_limit"],
    load_best_model_at_end=TRAINING_CONFIG["load_best_model_at_end"],
    metric_for_best_model=TRAINING_CONFIG["metric_for_best_model"],
    greater_is_better=TRAINING_CONFIG["greater_is_better"],

    # Other settings
    seed=TRAINING_CONFIG["seed"],
    fp16=TRAINING_CONFIG["fp16"],
    dataloader_num_workers=TRAINING_CONFIG["dataloader_num_workers"],
    report_to=TRAINING_CONFIG["report_to"],
    push_to_hub=TRAINING_CONFIG["push_to_hub"],

    # Remove unused columns
    remove_unused_columns=False,
)

print("✅ Training arguments created successfully!")
print(f"Output directory: {training_args.output_dir}")
print(f"Total training steps will be calculated after loading dataset")

✅ Training arguments created successfully!
Output directory: /content/drive/MyDrive/finetuned_finbert/model_output
Total training steps will be calculated after loading dataset


In [32]:
# Define metrics for evaluation
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# Load metric from evaluate library
metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    """
    Compute metrics for evaluation.

    Parameters:
    - eval_pred: tuple of predictions and labels

    Returns:
    - Dictionary of metrics
    """
    predictions, labels = eval_pred

    # Get predicted class (argmax)
    predictions = np.argmax(predictions, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")  # Weighted F1 for imbalanced datasets
    f1_macro = f1_score(labels, predictions, average="macro")  # Macro F1
    precision = precision_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")

    return {
        "accuracy": accuracy,
        "f1": f1,
        "f1_macro": f1_macro,
        "precision": precision,
        "recall": recall,
    }

print("✅ Metrics function defined successfully!")
print("Metrics: accuracy, f1 (weighted), f1_macro, precision, recall")

✅ Metrics function defined successfully!
Metrics: accuracy, f1 (weighted), f1_macro, precision, recall


In [33]:
# Load tokenized datasets from Google Drive
from datasets import load_from_disk

# Load datasets
dataset_dict = load_from_disk(TOKENIZED_DATA_DIR)

print("=" * 60)
print("DATASET LOADED")
print("=" * 60)
print(f"Train: {len(dataset_dict['train'])} samples")
print(f"Validation: {len(dataset_dict['validation'])} samples")
print(f"Test: {len(dataset_dict['test'])} samples")
print(f"Features: {dataset_dict['train'].features}")

# Calculate total training steps
total_steps = len(dataset_dict['train']) // TRAINING_CONFIG["per_device_train_batch_size"] * TRAINING_CONFIG["num_train_epochs"]
print(f"\nTotal training steps: {total_steps}")
print(f"Warmup steps: {TRAINING_CONFIG['warmup_steps']}")
print(f"Warmup ratio: {TRAINING_CONFIG['warmup_steps'] / total_steps * 100:.2f}%")
print("=" * 60)

DATASET LOADED
Train: 18391 samples
Validation: 2299 samples
Test: 2299 samples
Features: {'input_ids': List(Value('int32')), 'token_type_ids': List(Value('int8')), 'attention_mask': List(Value('int8')), 'labels': Value('int64')}

Total training steps: 5745
Warmup steps: 500
Warmup ratio: 8.70%


In [34]:
# Verify all configurations
print("=" * 60)
print("TRAINING CONFIGURATION SUMMARY")
print("=" * 60)

print(f"\nModel:")
print(f"  Name: {MODEL_NAME}")
print(f"  Number of labels: {NUM_LABELS}")
print(f"  Label mapping: {label_to_id}")

print(f"\nHyperparameters:")
print(f"  Learning rate: {TRAINING_CONFIG['learning_rate']}")
print(f"  Epochs: {TRAINING_CONFIG['num_train_epochs']}")
print(f"  Batch size (train): {TRAINING_CONFIG['per_device_train_batch_size']}")
print(f"  Batch size (eval): {TRAINING_CONFIG['per_device_eval_batch_size']}")
print(f"  Weight decay: {TRAINING_CONFIG['weight_decay']}")
print(f"  Warmup steps: {TRAINING_CONFIG['warmup_steps']}")

print(f"\nOptimizer & Scheduler:")
print(f"  Optimizer: {TRAINING_CONFIG['optim']}")
print(f"  LR Scheduler: {TRAINING_CONFIG['lr_scheduler_type']}")

print(f"\nLoss Function:")
print(f"  Type: CrossEntropyLoss (for classification)")
print(f"  Problem type: single_label_classification")

print(f"\nTraining Settings:")
print(f"  Output directory: {TRAINING_CONFIG['output_dir']}")
print(f"  Logging directory: {TRAINING_CONFIG['logging_dir']}")
print(f"  Evaluation strategy: {TRAINING_CONFIG['eval_strategy']}")
print(f"  Save strategy: {TRAINING_CONFIG['save_strategy']}")
print(f"  FP16: {TRAINING_CONFIG['fp16']}")

print(f"\nDataset:")
print(f"  Train samples: {len(dataset_dict['train'])}")
print(f"  Validation samples: {len(dataset_dict['validation'])}")
print(f"  Test samples: {len(dataset_dict['test'])}")

print(f"\nGPU:")
print(f"  Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  Device: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

print("\n✅ All configurations verified!")
print("=" * 60)

TRAINING CONFIGURATION SUMMARY

Model:
  Name: ProsusAI/finbert
  Number of labels: 3
  Label mapping: {np.int64(0): 0, np.int64(1): 1, np.int64(2): 2}

Hyperparameters:
  Learning rate: 2e-05
  Epochs: 5
  Batch size (train): 16
  Batch size (eval): 16
  Weight decay: 0.01
  Warmup steps: 500

Optimizer & Scheduler:
  Optimizer: adamw_torch
  LR Scheduler: linear

Loss Function:
  Type: CrossEntropyLoss (for classification)
  Problem type: single_label_classification

Training Settings:
  Output directory: /content/drive/MyDrive/finetuned_finbert/model_output
  Logging directory: /content/drive/MyDrive/finetuned_finbert/logs
  Evaluation strategy: epoch
  Save strategy: epoch
  FP16: True

Dataset:
  Train samples: 18391
  Validation samples: 2299
  Test samples: 2299

GPU:
  Available: True
  Device: Tesla T4
  Memory: 15.83 GB

✅ All configurations verified!


In [35]:
# Create data collator for dynamic padding
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding=True,
    max_length=512,
    return_tensors="pt"
)

print("✅ Data collator created successfully!")

✅ Data collator created successfully!


In [36]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✅ Trainer initialized successfully!")
print(f"Train dataset: {len(trainer.train_dataset)} samples")
print(f"Eval dataset: {len(trainer.eval_dataset)} samples")

✅ Trainer initialized successfully!
Train dataset: 18391 samples
Eval dataset: 2299 samples


In [37]:
# Train the model
print("=" * 60)
print("STARTING TRAINING")
print("=" * 60)
print(f"Training for {TRAINING_CONFIG['num_train_epochs']} epochs...")
print(f"Total training steps: {len(trainer.get_train_dataloader()) * TRAINING_CONFIG['num_train_epochs']}")
print("=" * 60)

# Train
train_results = trainer.train()

print("\n" + "=" * 60)
print("TRAINING COMPLETED")
print("=" * 60)
print(f"Training loss: {train_results.training_loss:.4f}")
print(f"Training runtime: {train_results.metrics['train_runtime']:.2f} seconds")
print(f"Training samples per second: {train_results.metrics['train_samples_per_second']:.2f}")
print("=" * 60)

STARTING TRAINING
Training for 5 epochs...
Total training steps: 5750


Epoch,Training Loss,Validation Loss,Accuracy,F1,F1 Macro,Precision,Recall
1,0.414800,0.386664,0.844715,0.847211,0.824525,0.855875,0.844715
2,0.290300,0.374679,0.859504,0.860448,0.840149,0.863986,0.859504
3,0.162800,0.479578,0.859504,0.861390,0.837192,0.867090,0.859504
4,0.113700,0.512045,0.872988,0.874771,0.849952,0.879772,0.872988
5,0.074500,0.584438,0.869943,0.871142,0.846954,0.876234,0.869943



TRAINING COMPLETED
Training loss: 0.2366
Training runtime: 2411.42 seconds
Training samples per second: 38.13
